# setup

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import bokeh.plotting as bpl
import cv2
import glob
import logging
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')
import numpy as np
import os
import pandas as pd
import pickle
from IPython.display import clear_output
try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
bpl.output_notebook()

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
#                     filename="/tmp/caiman.log",
                    level=logging.INFO)

Loading BokehJS ...

# files & functions

In [39]:
def caiman_parameter_set(file_now):
    fr = 30                             # imaging rate in frames per second
    decay_time = 0.8                    # length of a typical transient in seconds
    # revised to account for gcamp6s, ~750 ms decay time in V1 L2/3: https://elifesciences.org/articles/51675
    # decay_time: Length of a typical transient in seconds. decay_time is an approximation of the time scale over which to expect a significant shift in the calcium signal during a transient. It defaults to 0.4, which is appropriate for fast indicators (GCaMP6f), slow indicators might use 1 or even more. However, decay_time does not have to precisely fit the data, approximations are enough.
    # V1 layer 2/3: rise and decay time constants for GCaMP6f are 50-100 ms and 200-300 ms; and for GCaMP6s 150-200 ms and ~750 ms (4-5 APs, Figure 3C). https://elifesciences.org/articles/51675

    # motion correction parameters
    strides = (48, 48)          # start a new patch for pw-rigid motion correction every x pixels
    overlaps = (24, 24)         # overlap between pathes (size of patch strides+overlaps)
    max_shifts = (6,6)          # maximum allowed rigid shifts (in pixels)
    max_deviation_rigid = 3     # maximum shifts deviation allowed for patch with respect to rigid shifts
    pw_rigid = True             # flag for performing non-rigid motion correction

    # parameters for source extraction and deconvolution
    p = 1                       # order of the autoregressive system
    gnb = 2                     # number of global background components
    merge_thr = 0.85            # merging threshold, max correlation allowed
    rf = 15                     # half-size of the patches in pixels. e.g., if rf=25, patches are 50x50
    stride_cnmf = 6             # amount of overlap between the patches in pixels
    K = 4                       # number of components per patch
    gSig = [4, 4]               # expected half size of neurons in pixels 
    # kept 4 according to snipaste pixel measurement
    # adjust cell size bc of zoom change around 2021-09

    method_init = 'greedy_roi'  # initialization method (if analyzing dendritic data using 'sparse_nmf')
    ssub = 1                    # spatial subsampling during initialization
    tsub = 1                    # temporal subsampling during intialization

    # parameters for component evaluation
    min_SNR = 2.0               # signal to noise ratio for accepting a component
    rval_thr = 0.90             # space correlation threshold for accepting a component # adjusted
    rval_lowest = 0.0           # added to eliminate fake overlapping neurons, default -1
    cnn_thr = 0.99              # threshold for CNN based classifier
    cnn_lowest = 0.50           # neurons with cnn probability lower than this value are rejected # adjusted
    
    opts_dict = {'fnames': file_now,
                'fr': fr,
                'decay_time': decay_time,
                'strides': strides,
                'overlaps': overlaps,
                'max_shifts': max_shifts,
                'max_deviation_rigid': max_deviation_rigid,
                'pw_rigid': pw_rigid,
                'p': p,
                'nb': gnb,
                'rf': rf,
                'K': K, 
                'stride': stride_cnmf,
                'method_init': method_init,
                'rolling_sum': True,
                'only_init': True,
                'ssub': ssub,
                'tsub': tsub,
                'merge_thr': merge_thr, 
                'min_SNR': min_SNR,
                'rval_thr': rval_thr,
                'use_cnn': True,
                'min_cnn_thr': cnn_thr,
                'cnn_lowest': cnn_lowest}

    opts = params.CNMFParams(params_dict=opts_dict)
    
    return opts

In [40]:
# def cluster_start():
#     #%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
#     if 'dview' in locals():
#         cm.stop_server(dview=dview)
#     c, dview, n_processes = cm.cluster.setup_cluster(
#         backend='local', n_processes=None, single_thread=False)

In [41]:
def caiman_motion_correct_memmap(mc):
    %%capture
    #%% Run piecewise-rigid motion correction using NoRMCorre
    mc.motion_correct(save_movie=True)
    m_els = cm.load(mc.fname_tot_els)
    border_to_0 = 0 if mc.border_nan is 'copy' else mc.border_to_0 # maximum shift to be used for trimming against NaNs
    
    #%% MEMORY MAPPING
    fname_new = cm.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                               border_to_0=border_to_0, dview=dview) # exclude borders
    Yr, dims, T = cm.load_memmap(fname_new)
    images = np.reshape(Yr.T, [T] + list(dims), order='F') #load frames in python format (T x X x Y)
    
    #%% restart cluster to clean up memory
    cm.stop_server(dview=dview)
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)
    
    return images, fname_new, c, dview, n_processes

In [42]:
def caiman_fit(n_processes, opts, dview, images):
    %%capture
    #%% RUN CNMF ON PATCHES
    # First extract spatial and temporal components on patches and combine them
    # for this step deconvolution is turned off (p=0). If you want to have
    # deconvolution within each patch change params.patch['p_patch'] to a
    # nonzero value
    cnm = cnmf.CNMF(n_processes, params=opts, dview=dview)
    cnm = cnm.fit(images)
    
    return cnm

In [43]:
def corr_img_save(file_now, images):
    Cn = cm.local_correlations(images.transpose(1,2,0))
    f = open(file_now[:-4] + 'Cn.pckl', 'wb')
    pickle.dump(Cn, f)
    f.close()
    
    return Cn

In [44]:
caiman_data_path = 'C:/Users/ll357/Documents/CaImAn/demos/temp_data/'
tif_files = glob.glob(caiman_data_path + 'i1350_211222/' + "*70k.tif", recursive = True)
tif_files

['C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\\i1350_211222_002_multipage_70k.tif',
 'C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\\i1350_211222_003_multipage_70k.tif',
 'C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\\i1350_211222_004_multipage_70k.tif']

# run

In [17]:
# cnm1.fit_file(motion_correct=False, include_eval=True)

    82423360 [cnmf.py:                 fit():470] [41556] (70000, 264, 796)
    82423360 [cnmf.py:                 fit():487] [41556] Using 63 processes
    82423360 [cnmf.py:                 fit():498] [41556] using 2070 pixels per process
    82423360 [cnmf.py:                 fit():499] [41556] using 5000 block_size_spat
    82423360 [cnmf.py:                 fit():500] [41556] using 5000 block_size_temp
    82423360 [params.py:                 set():972] [41556] Changing key n_pixels_per_process in group preprocess from 2070 to 225
    82423366 [params.py:                 set():972] [41556] Changing key n_pixels_per_process in group spatial from 2070 to 225
    82423391 [map_reduce.py:    run_CNMF_patches():245] [41556] Patch size: (30, 30)
    85636606 [map_reduce.py:    run_CNMF_patches():264] [41556] Elapsed time for processing patches:                  3213s
    85636649 [map_reduce.py:    run_CNMF_patches():325] [41556] Embedding patches results into whole FOV
    85637287 [ma

    85674704 [merging.py:    merge_components():238] [41556] Merging components [1209 1341]
    85675133 [merging.py:    merge_components():238] [41556] Merging components [961 964]
    85675564 [merging.py:    merge_components():238] [41556] Merging components [1192 1324]
    85675983 [merging.py:    merge_components():238] [41556] Merging components [1312 1444]
    85676401 [merging.py:    merge_components():238] [41556] Merging components [69 72]
    85676810 [merging.py:    merge_components():238] [41556] Merging components [604 611]
    85677249 [merging.py:    merge_components():238] [41556] Merging components [1224 1356]
    85677672 [merging.py:    merge_components():238] [41556] Merging components [103 234]
    85678127 [merging.py:    merge_components():238] [41556] Merging components [932 937]
    85678574 [merging.py:    merge_components():238] [41556] Merging components [828 960]
    85678985 [merging.py:    merge_components():238] [41556] Merging components [776 780]
    

    85713393 [merging.py:    merge_components():238] [41556] Merging components [88 94]
    85713813 [merging.py:    merge_components():238] [41556] Merging components [22 25]
    85714238 [merging.py:    merge_components():238] [41556] Merging components [ 976 1111]
    85714677 [merging.py:    merge_components():238] [41556] Merging components [442 572]
    85715095 [merging.py:    merge_components():238] [41556] Merging components [1281 1415]
    85715493 [merging.py:    merge_components():238] [41556] Merging components [944 951]
    85715900 [merging.py:    merge_components():238] [41556] Merging components [500 634]
    85716316 [merging.py:    merge_components():238] [41556] Merging components [576 582]
    85716730 [merging.py:    merge_components():238] [41556] Merging components [704 837]
    85717185 [merging.py:    merge_components():238] [41556] Merging components [699 703]
    85717579 [merging.py:    merge_components():238] [41556] Merging components [476 482]
    857180

    85752012 [merging.py:    merge_components():238] [41556] Merging components [870 874]
    85752419 [merging.py:    merge_components():238] [41556] Merging components [226 358]
    85752838 [merging.py:    merge_components():238] [41556] Merging components [1088 1092]
    85753296 [merging.py:    merge_components():238] [41556] Merging components [ 887 1017]
    85753719 [merging.py:    merge_components():238] [41556] Merging components [681 812]
    85754127 [merging.py:    merge_components():238] [41556] Merging components [ 67 198]
    85754590 [merging.py:    merge_components():238] [41556] Merging components [313 318]
    85755024 [merging.py:    merge_components():238] [41556] Merging components [831 833]
    85755422 [merging.py:    merge_components():238] [41556] Merging components [172 177]
    85755840 [merging.py:    merge_components():238] [41556] Merging components [1032 1167]
    85756274 [merging.py:    merge_components():238] [41556] Merging components [904 908]
    

IndexError: index 1003 is out of bounds for axis 0 with size 999

In [7]:
# for file_now in tif_files:
#     opts = caiman_parameter_set(file_now)
#     clear_output(wait=True)
#     print(file_now[:-4])
    
    
#     c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)
#     print('start fitting caiman')
#     mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

    
# #     cnm1 = cnmf.CNMF(n_processes, params=opts, dview=dview)
# #     cnm1.fit_file(motion_correct=True)
# #     cnm1.fit_file(motion_correct=False, indices=None, include_eval=True)
    
#     corr_img_save(file_now) # images not defined
#     cnm1.estimates.evaluate_components(images, cnm2.params, dview=dview)
#     print(len(cnm1.estimates.idx_components))
#     print(len(cnm1.estimates.idx_components_bad))
#     cnm1.estimates.detrend_df_f(quantileMin=8, frames_window=250) # generates cnm.F_dff
    
#     cnm1.save(file_now[:-4] + '.hdf5')
#     cm.stop_server(dview=dview)

C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\i1350_211222_002_multipage_70k


      204215 [params.py:                 set():972] [41556] Changing key n_processes in group patch from 1 to 63


start fitting caiman


      205435 [motion_correction.py:motion_correct_pwrigid():355] [41556] Generating template by rigid motion correction
      283948 [motion_correction.py:motion_correction_piecewise():3149] [41556] ** Starting parallel motion correction **
     3649804 [motion_correction.py:motion_correction_piecewise():3157] [41556] ** Finished parallel motion correction **
     3662449 [motion_correction.py:motion_correction_piecewise():3136] [41556] Saving file as C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\i1350_211222_002_multipage_70k_els__d1_264_d2_796_d3_1_order_F_frames_70000_.mmap
     3662614 [motion_correction.py:motion_correction_piecewise():3149] [41556] ** Starting parallel motion correction **
    37869285 [motion_correction.py:motion_correction_piecewise():3157] [41556] ** Finished parallel motion correction **
    38170385 [params.py:                 set():972] [41556] Changing key init_batch in group online from 200 to 70000
    38170385 [cnmf.py:                 fi

    41653475 [merging.py:    merge_components():238] [41556] Merging components [160 290 294]
    41653929 [merging.py:    merge_components():238] [41556] Merging components [1159 1163 1294]
    41654331 [merging.py:    merge_components():238] [41556] Merging components [ 91 221 224]
    41654762 [merging.py:    merge_components():238] [41556] Merging components [ 868  873 1006]
    41655202 [merging.py:    merge_components():238] [41556] Merging components [354 357 489]
    41655604 [merging.py:    merge_components():238] [41556] Merging components [742 869 872]
    41656037 [merging.py:    merge_components():238] [41556] Merging components [1300 1304 1433]
    41656465 [merging.py:    merge_components():238] [41556] Merging components [1180 1311 1313]
    41656910 [merging.py:    merge_components():238] [41556] Merging components [1056 1062 1188]
    41657325 [merging.py:    merge_components():238] [41556] Merging components [ 19  23 153]
    41657730 [merging.py:    merge_components

    41692518 [merging.py:    merge_components():238] [41556] Merging components [205 336]
    41692942 [merging.py:    merge_components():238] [41556] Merging components [1281 1415]
    41693366 [merging.py:    merge_components():238] [41556] Merging components [844 978]
    41693805 [merging.py:    merge_components():238] [41556] Merging components [167 298]
    41694226 [merging.py:    merge_components():238] [41556] Merging components [333 337]
    41694645 [merging.py:    merge_components():238] [41556] Merging components [820 952]
    41695075 [merging.py:    merge_components():238] [41556] Merging components [1342 1346]
    41695521 [merging.py:    merge_components():238] [41556] Merging components [344 477]
    41695947 [merging.py:    merge_components():238] [41556] Merging components [ 892 1024]
    41696356 [merging.py:    merge_components():238] [41556] Merging components [154 285]
    41696783 [merging.py:    merge_components():238] [41556] Merging components [564 698]
    

    41731831 [merging.py:    merge_components():238] [41556] Merging components [537 668]
    41732242 [merging.py:    merge_components():238] [41556] Merging components [1161 1164]
    41732670 [merging.py:    merge_components():238] [41556] Merging components [1238 1371]
    41733099 [merging.py:    merge_components():238] [41556] Merging components [877 881]
    41733531 [merging.py:    merge_components():238] [41556] Merging components [792 925]
    41733973 [merging.py:    merge_components():238] [41556] Merging components [1044 1178]
    41734385 [merging.py:    merge_components():238] [41556] Merging components [856 861]
    41734807 [merging.py:    merge_components():238] [41556] Merging components [708 712]
    41735217 [merging.py:    merge_components():238] [41556] Merging components [979 980]
    41735651 [merging.py:    merge_components():238] [41556] Merging components [700 834]
    41736074 [merging.py:    merge_components():238] [41556] Merging components [479 480]
    

    41880124 [temporal.py:    update_iteration():391] [41556] 956 out of total 999 temporal components updated
    41883796 [temporal.py:    update_iteration():391] [41556] 998 out of total 999 temporal components updated
    41884140 [temporal.py:    update_iteration():391] [41556] 999 out of total 999 temporal components updated
    41933046 [temporal.py:    update_iteration():391] [41556] 437 out of total 999 temporal components updated
    41955231 [temporal.py:    update_iteration():391] [41556] 778 out of total 999 temporal components updated
    41971037 [temporal.py:    update_iteration():391] [41556] 956 out of total 999 temporal components updated
    41975035 [temporal.py:    update_iteration():391] [41556] 998 out of total 999 temporal components updated
    41975396 [temporal.py:    update_iteration():391] [41556] 999 out of total 999 temporal components updated


NameError: name 'images' is not defined

In [61]:
str(type(dview))

"<class 'multiprocessing.pool.Pool'>"

In [60]:
for file_now in tif_files:
    
    opts = caiman_parameter_set(file_now)
    clear_output(wait=True)
    print('param set')
    print(file_now[:-4])
    
#     cm.stop_server(dview=dview)
#     dview.terminate()
#     c, dview, n_processes = cm.cluster.setup_cluster(backend='local', n_processes=None, single_thread=False)

    try:
        cm.stop_server()
        dview.terminate()
    except:
        print('No clusters to stop')
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='multiprocessing', n_processes=None, single_thread=False)
    print('cluster started')
    
    mc = MotionCorrect(file_now, dview=dview, **opts.get_group('motion'))
    images, fname_new, c, dview, n_processes = caiman_motion_correct_memmap(mc)
    print('motion corrected')
    
    cnm1 = caiman_fit(n_processes, opts, dview, images)
    print('caiman fitted')
    
    Cn = corr_img_save(file_now, images)
    print('correlation image done')
    cnm1.estimates.evaluate_components(images, cnm2.params, dview=dview)
    print(len(cnm1.estimates.idx_components))
    print(len(cnm1.estimates.idx_components_bad))
    cnm1.estimates.detrend_df_f(quantileMin=8, frames_window=250) # generates cnm.F_dff
    print('dff calculated')
    
    cnm1.save(file_now[:-4] + '.hdf5')
    cm.stop_server(dview=dview)

   171863494 [cluster.py:         stop_server():291] [41556] Stopping cluster...


param set
C:/Users/ll357/Documents/CaImAn/demos/temp_data/i1350_211222\i1350_211222_002_multipage_70k


   171864146 [cluster.py:         stop_server():334] [41556] No cluster to stop...
   171864147 [cluster.py:         stop_server():348] [41556] stop_cluster(): done


Exception: A cluster is already runnning. Terminate with dview.terminate() if you want to restart.